# Machine Learning
This notebook houses tasks related to


In [172]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline

In [173]:
data_dir = "data/processed/"

# Import precinct data
df_precincts = pd.read_csv(data_dir + "precincts_with_registration.csv")
df_precincts.drop("Unnamed: 0", axis=1, inplace=True)

# Import election total data
df_election_totals = pd.read_csv(data_dir + "election_totals.csv")
df_election_totals.set_index("year", inplace=True)

df_registration_totals = pd.read_csv(data_dir + "registration_totals.csv")
df_registration_totals.set_index("year", inplace=True)

df_elections_with_registration = df_election_totals.join(df_registration_totals, how="inner")
df_elections_with_registration


,ABSVOTE,AIPREG,AIPVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSPAF01,ASSREP01,ASSREP02,CNGDEM01,...,rreg5g,rreg6g,rreg7g,rreg8g,rreg9g,totreg_r,vietdcl,vietdem,vietoth,vietrep
year,,,,,,,,,,,,,,,,,,,,,
2012,0.0,0.0,0.0,65500.0,0.0,0.0,0.0,103762.0,0.0,86907.0,...,10967.0,4656.0,5996.0,3922.0,23554.0,219403.0,70.0,82.0,18.0,89.0
2014,0.0,0.0,0.0,46126.0,0.0,0.0,NaN,77452.0,0.0,59214.0,...,4937.0,9098.0,3954.0,5177.0,24026.0,216298.0,93.0,94.0,24.0,80.0
2016,0.0,0.0,0.0,87168.0,0.0,0.0,NaN,105247.0,0.0,89992.0,...,6849.0,3395.0,7910.0,3179.0,25474.0,249413.0,143.0,174.0,21.0,92.0
2018,0.0,0.0,0.0,56257.0,0.0,0.0,NaN,67443.0,NaN,91599.0,...,3537.0,4856.0,2506.0,5984.0,22393.0,248773.0,158.0,165.0,23.0,93.0
2020,0.0,0.0,0.0,103206.0,0.0,0.0,NaN,126579.0,0.0,120883.0,...,3025.0,1985.0,2737.0,1425.0,17286.0,279110.0,164.0,232.0,51.0,152.0


## Using Party Registration to Predict Democratic Candidate Performance


In [174]:
# Normalize party registration per year.
df_elections_with_registration[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]] = \
    df_elections_with_registration[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]].div(df_elections_with_registration["totreg_r"], axis=0)

# Normalize assembly candidate performance
df_elections_with_registration[["ASSDEM01", "ASSREP01"]] = \
    df_elections_with_registration[["ASSDEM01", "ASSREP01"]].div(df_elections_with_registration[["ASSDEM01", "ASSREP01"]].sum(axis=1), axis=0)

df_elections_with_registration[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]]

print("Actual Results: ", df_elections_with_registration["ASSDEM01"])

Actual Results:  year
2012    0.386974
2014    0.373254
2016    0.453021
2018    0.454786
2020    0.449142
Name: ASSDEM01, dtype: float64


In [175]:
df_precincts[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]] = \
    df_precincts[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]].div(df_precincts["totreg_r"], axis=0)

# Normalize assembly candidate performance
df_precincts[["ASSDEM01", "ASSREP01"]] = \
    df_precincts[["ASSDEM01", "ASSREP01"]].div(df_precincts[["ASSDEM01", "ASSREP01"]].sum(axis=1), axis=0)

In [176]:
train_labels = ["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]
X_train = df_precincts[train_labels]
X_test = df_elections_with_registration[train_labels]

y_train = df_precincts["ASSDEM01"]
y_test = df_elections_with_registration["ASSDEM01"]

pipeline = make_pipeline(
#    StandardScaler(),
    KNeighborsRegressor(n_neighbors=20)
)

pipeline.fit(X=X_train, y=y_train)

_y_test = pipeline.predict(X=X_test)
df_pred = pd.DataFrame(index=["2012", "2014", "2016", "2018", "2020"], data=_y_test)

print("Predicted Results:", df_pred)
print("Mean Squared Error: {}".format(mean_squared_error(_y_test, y_test)))
print("MPE: {}".format(mean_absolute_percentage_error(_y_test, y_test)))
print("MAE: {}".format(mean_absolute_error(_y_test, y_test)))


Predicted Results:              0
2012  0.392275
2014  0.397045
2016  0.411970
2018  0.426710
2020  0.438199
Mean Squared Error: 0.000637456747098332
MPE: 0.05276921185735393
MAE: 0.021832203608864943


### Finding Scenarios That Would Predict Democratic Gains

1. Say Democrats registered 2.5 percent of DCL's, to roughly match the average registraion percents of Republicans. What would the Democrat's predicted share of the vote be?

In [177]:
# Assume we start from the previous election registration.
prev_performance = df_elections_with_registration.loc[2020]

high_dem_registration = prev_performance[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]]

high_dem_registration["dem"] = high_dem_registration["dem"] + 0.025
high_dem_registration["dcl"] = high_dem_registration["dcl"] - 0.025

pred = pipeline.predict(X=[high_dem_registration])

print("2.5 point gain predicted results: {}%".format(pred[0]))

high_dem_registration["dem"] = high_dem_registration["dem"] + 0.05
high_dem_registration["dcl"] = high_dem_registration["dcl"] - 0.05

pred = pipeline.predict(X=[high_dem_registration])
print("3 point gain predicted results: {}%".format(pred[0])

2.5 point gain predicted results: 0.48107987558758003%


array([0.50518681])

Answer: Not quite enough. 

## Using Voter Demographics To Predict Democratic Candidate Performance


In [178]:
df_precincts[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]] = \
    df_precincts[["dem","rep","aip","paf","msc","lib","nlp","grn","ref","dcl"]].div(df_precincts["totreg_r"], axis=0)

# Normalize assembly candidate performance
df_precincts[["ASSDEM01", "ASSREP01"]] = \
    df_precincts[["ASSDEM01", "ASSREP01"]].div(df_precincts[["ASSDEM01", "ASSREP01"]].sum(axis=1), axis=0)

## Finding Model Precincts